# Figshare API

In [1]:
#import libraries
import requests as rq
import pandas as pd
from pprint import pprint as pp
import pickle

API access tokens have been stored in credentials.pkl file

In [2]:
#import API access token
#at the moment, only have a single token in my credentials.pkl, which isn't the long term plan - need to change this
FIGSHARE_TOKEN = pickle.load(open('credentials.pkl', 'rb'))

In [3]:
#Search seems to be all by type (articles, collections, etc.)
#All are public
#Plan is to search (1) articles, (2) collections, (3) projects
#Article search may be minimally useful, but if have linked DOIs in object, could be linked data

#Can set up to cycle through which search (articles, collections, data) and by page

Overall workflow: use main search to get IDs of matching objects, then use IDs to get full object details & associated files

## Code that loops through search terms and objects
## but does not include pagination
### also an issue with projects objects that stops code

In [23]:
#FIGSHARE_TOKEN imported from credentials.pkl
HEADERS = {'Authorization': 'token '+ FIGSHARE_TOKEN}

#List of search terms
SEARCH_TERMS = ['machine learning', 'artificial intelligence']

###### LOOP 1 - loop through search terms ######
for k in SEARCH_TERMS:
    search_k = k
    
    print('Now on search term ' + search_k)

    #Specify page to return with search (results are paginated) - TO DO: FIX HARD CODING
    PAGE = 1
    
    #Specify number of results included on a page (default is 10, max is 1000) - TO DO: FIX HARD CODING
    PAGE_SIZE = 10

    #Set params term
    PARAMS = {
        'search_for': search_k, #search term
        'page': PAGE, 
        'page_size': PAGE_SIZE,  
        }

    #List of Figshare object types to search
    SEARCH_TYPES = ['articles', 'collections', 'projects']
    
    ###### LOOP 2 - loop through object types ######

    for j in SEARCH_TYPES:
        
        print('For object type ' + j)
        
        URL_j = "https://api.figshare.com/v2/" + j

        ## Run search for public articles
        response = rq.get(URL_j, params=PARAMS, headers=HEADERS)
    
        ## Put output into json format
        output = response.json()
    
        #Convert output to pd dataframe and see table format
        df_full = pd.DataFrame(output)

        ## Extract IDs
        full_ids = list(df_full.id)

        ####### LOOP 3 - loop to to extract article details by object ID ######

        #create empty pandas dataframe to put results in
        df_detailed = pd.DataFrame()

        #for each ID, get full object details
        for i in full_ids:
            id_i = str(i)
            #URL syntax for object details is: https://api.figshare.com/v2/{search_type}/{object_id}
            URL_i = URL_j + '/' + id_i
            response_i = rq.get(URL_i, headers=HEADERS)
            json_i = response_i.json()
            #json_normalize collapses first level, which is a start
            #for now, can leave files, custom fields, author, etc as list of dictionary
            df_i = pd.json_normalize(json_i)
            df_detailed = df_detailed.append(df_i)

            #Keep just id and timeline from initial extract (all others are in detailed extract)
            #'timeline' is NOT in projects, so...do we really need it?
            df_small = df_full[['id','timeline']]

            #Add info about which search type and page this is associated with
            df_detailed['Search_type'] = j
            df_detailed['search_page'] = PAGE

            #Merge small version of "full" dataframe with "detailed" dataframe
            df_all = pd.merge(df_small, df_detailed, on = "id", how = "inner")

            #Write to csv
            file_name_jk = "Figshare_" + k + '_' + j +'.csv'
            df_all.to_csv(file_name_jk, index=False)
            
    print('Finished ' + j + ' ' + k)

Now on search termmachine learning
For object typearticles
For object typecollections
For object typeprojects


KeyError: "['timeline'] not in index"

## OLD CODE
#### Specify search parameters

In [6]:
#FIGSHARE_TOKEN imported from credentials.pkl
HEADERS = {'Authorization': 'token '+ FIGSHARE_TOKEN}

#Specify search terms
SEARCH = 'machine learning'

#turn into list for loop: SEARCH = ['machine learning'] 
#with whatever search terms we decide to use

#Specify which search (collections, articles, projects, data)
ARTICLE_URL = 'https://api.figshare.com/v2/articles'
COLLECTIONS_URL = "https://api.figshare.com/v2/collections"
PROJECTS_URL = "https://api.figshare.com/v2/projects"

#turn into list for loop: SEARCH_TYPES = ['articles', 'collections', 'projects']
#for j in SEARCH_TYPES:
    #URL_j = "https://api.figshare.com/v2/" + SEARCH_TYPES_j
    
#article search should include datasets
#Only return articles with the respective type. Mapping for item_type is: 
#1 - Figure, 2 - Media, 3 - Dataset, 5 - Poster, 6 - Journal contribution, 7 - Presentation, 
#8 - Thesis, 9 - Software, 11 - Online resource, 12 - Preprint, 13 - Book, 14 - Conference contribution, 
#15 - Chapter, 16 - Peer review, 17 - Educational resource, 18 - Report, 19 - Standard, 20 - Composition, 
#21 - Funding, 22 - Physical object, 23 - Data management plan, 24 - Workflow, 
#25 - Monograph, 26 - Performance, 27 - Event, 28 - Service, 29 - Model

#should make a df with this and merge to get more informative item_type column in final output

#Specify page to return with search (results are paginated)
PAGE = 1

#Specify number of results included on a page (default is 10, max is 1000)
PAGE_SIZE = 10

## Could iterate through pages until get response: {'message': 'Bad Request', 'code': 'BadRequest'} 
## Seems like there should be a better way

#Specify page and page size parameters
#Other search options are available, including limit and offset, but at the moment page/page size seem most useful
    #if set both page/page size and limit/offset, get:
    #{'message': 'Pagination options can be set either via page/page_size or limit/offset params','code': 'ConflictingPaginationOptions'}
    
    #'limit': 1000, #Number of results included on a page. Used for pagination with query (optional) - not sure how differs from page_size
    #'offset': 1 #, #Where to start the listing(the offset of the first result). Used for pagination with limit (optional)
#We don't need to specify other search parameters like institution, group, modified since, etc.

#Full search parameters
PARAMS = {
    'search_for': SEARCH, #search term
    'page': PAGE, 
    'page_size': PAGE_SIZE,  
    }

## Draft workflow: search public articles

In [7]:
## Run search for public articles
response = rq.get(ARTICLE_URL, params=PARAMS, headers=HEADERS)

## Put output into json format
output = response.json()

#Convert output to pd dataframe and see table format
df_full = pd.DataFrame(output)

In [8]:
## See what output looks like
output

[{'id': 6716609,
  'title': 'Expressive Collaborative Music Performance via Machine Learning',
  'doi': '10.1184/r1/6716609.v1',
  'handle': '',
  'url': 'https://api.figshare.com/v2/articles/6716609',
  'published_date': '2016-08-01T00:00:00Z',
  'thumb': 'https://s3-eu-west-1.amazonaws.com/ppreviews-cmu-49810598590254/12249995/thumb.png',
  'defined_type': 8,
  'defined_type_name': 'thesis',
  'group_id': 18614,
  'url_private_api': 'https://api.figshare.com/v2/account/articles/6716609',
  'url_public_api': 'https://api.figshare.com/v2/articles/6716609',
  'url_private_html': 'https://figshare.com/account/articles/6716609',
  'url_public_html': 'https://kilthub.cmu.edu/articles/thesis/Expressive_Collaborative_Music_Performance_via_Machine_Learning/6716609',
  'timeline': {'posted': '2016-08-01T00:00:00',
   'firstOnline': '2018-07-01T00:04:06',
   'revision': '2018-07-01T00:04:07'},
  'resource_title': '',
  'resource_doi': ''},
 {'id': 6714890,
  'title': 'Attribute Learning using J

In [9]:
## See what df looks like
df_full

,id,title,doi,handle,url,published_date,thumb,defined_type,defined_type_name,group_id,url_private_api,url_public_api,url_private_html,url_public_html,timeline,resource_title,resource_doi
0,6716609,Expressive Collaborative Music Performance via...,10.1184/r1/6716609.v1,,https://api.figshare.com/v2/articles/6716609,2016-08-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6716609,https://figshare.com/account/articles/6716609,https://kilthub.cmu.edu/articles/thesis/Expres...,"{'posted': '2016-08-01T00:00:00', 'firstOnline...",,
1,6714890,Attribute Learning using Joint Human and Machi...,10.1184/r1/6714890.v1,,https://api.figshare.com/v2/articles/6714890,2012-08-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6714890,https://figshare.com/account/articles/6714890,https://kilthub.cmu.edu/articles/thesis/Attrib...,"{'posted': '2012-08-01T00:00:00', 'firstOnline...",,
2,6720908,On Learning from Collective Data,10.1184/r1/6720908.v1,,https://api.figshare.com/v2/articles/6720908,2013-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720908,https://figshare.com/account/articles/6720908,https://kilthub.cmu.edu/articles/thesis/On_Lea...,"{'posted': '2013-12-01T00:00:00', 'firstOnline...",,
3,6720416,Learning with Staleness,10.1184/r1/6720416.v1,,https://api.figshare.com/v2/articles/6720416,2018-03-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720416,https://figshare.com/account/articles/6720416,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2018-03-01T00:00:00', 'firstOnline...",,
4,6720836,New Paradigms and Optimality Guarantees in Sta...,10.1184/r1/6720836.v1,,https://api.figshare.com/v2/articles/6720836,2017-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720836,https://figshare.com/account/articles/6720836,https://kilthub.cmu.edu/articles/thesis/New_Pa...,"{'posted': '2017-12-01T00:00:00', 'firstOnline...",,
5,6720410,Learning with Limited Supervision by Input and...,10.1184/r1/6720410.v1,,https://api.figshare.com/v2/articles/6720410,2012-05-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720410,https://figshare.com/account/articles/6720410,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2012-05-01T00:00:00', 'firstOnline...",,
6,6720413,"Learning with Sparcity: Structures, Optimizati...",10.1184/r1/6720413.v1,,https://api.figshare.com/v2/articles/6720413,2013-07-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720413,https://figshare.com/account/articles/6720413,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2013-07-01T00:00:00', 'firstOnline...",,
7,7553882,Distribution and Histogram (DisH) Learning,10.1184/r1/7553882.v1,,https://api.figshare.com/v2/articles/7553882,2018-07-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/7...,https://api.figshare.com/v2/articles/7553882,https://figshare.com/account/articles/7553882,https://kilthub.cmu.edu/articles/thesis/Distri...,"{'posted': '2018-07-01T00:00:00', 'revision': ...",,
8,8336783,Gradient Descent for Non-convex Problems in Mo...,10.1184/r1/8336783.v1,,https://api.figshare.com/v2/articles/8336783,2019-06-27T15:57:04Z,https://s3-eu-west-1.amazonaws.com/pprev

In [10]:
## Extract IDs
full_ids = list(df_full.id)

In [11]:
## Loop to extract article details by object ID

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}

#create empty pandas dataframe
df_detailed = pd.DataFrame()

#for each ID, 
for i in full_ids:
    id_i = str(i)
    URL_i = 'https://api.figshare.com/v2/articles/' + id_i
    #print(URL_i)
    response_i = rq.get(URL_i, headers=HEADERS)
    json_i = response_i.json()
    #json_normalize collapses first level, which is a start
    #for now, can leave files, custom fields, author, etc as list of dictionary
    df_i = pd.json_normalize(json_i)
    df_detailed = df_detailed.append(df_i)

In [12]:
df_detailed

,files,custom_fields,authors,figshare_url,description,funding,funding_list,version,status,size,...,url_public_html,resource_title,resource_doi,license.value,license.name,license.url,timeline.posted,timeline.firstOnline,timeline.revision,timeline.submission
0,"[{'id': 12249995, 'name': 'Expressive Collabor...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5464304, 'full_name': 'Guangyu Xia', ...",https://kilthub.cmu.edu/articles/thesis/Expres...,<p>Techniques of Artificial Intelligence and H...,None,[],1,public,11070988,...,https://kilthub.cmu.edu/articles/thesis/Expres...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2016-08-01T00:00:00,2018-07-01T00:04:06,2018-07-01T00:04:07,NaN
0,"[{'id': 12247874, 'name': 'Attribute Learning ...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5364275, 'full_name': 'Edith L.M. Law'...",https://kilthub.cmu.edu/articles/thesis/Attrib...,<p>This thesis is centered around the problem ...,None,[],1,public,8863551,...,https://kilthub.cmu.edu/articles/thesis/Attrib...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-08-01T00:00:00,2018-06-30T23:16:04,2018-06-30T23:16:05,NaN
0,"[{'id': 12255338, 'name': 'On Learning from Co...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5363849, 'full_name': 'Liang Xiong', ...",https://kilthub.cmu.edu/articles/thesis/On_Lea...,In many machine learning problems and applicat...,None,[],1,public,4152506,...,https://kilthub.cmu.edu/articles/thesis/On_Lea...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-12-01T00:00:00,2018-07-01T00:48:03,2018-07-01T00:48:04,NaN
0,"[{'id': 12254780, 'name': 'Learning with Stale...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4947985, 'full_name': 'Wei Dai', 'is_a...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>A fundamental assumption behind most machin...,None,[],1,public,4664211,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-03-01T00:00:00,2018-07-01T00:25:52,2018-07-01T00:25:53,NaN
0,"[{'id': 12255260, 'name': 'New Paradigms and O...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5466287, 'full_name': 'Yu-Xiang Wang'...",https://kilthub.cmu.edu/articles/thesis/New_Pa...,Machine learning (ML) has become one of the mo...,None,[],1,public,8304735,...,https://kilthub.cmu.edu/articles/thesis/New_Pa...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2017-12-01T00:00:00,2018-07-01T00:44:53,2018-07-01T00:44:54,NaN
0,"[{'id': 12254774, 'name': 'Learning with Limit...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5360768, 'full_name': 'Yi Zhang', 'is...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>In many real-world applications of supervis...,None,[],1,public,1611768,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-05-01T00:00:00,2018-07-01T00:25:35,2018-07-01T00:25:36,NaN
0,"[{'id': 12254777, 'name': 'Learning with Sparc...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362001, 'full_name': 'Xi Chen', 'is_...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>The development of modern information techn...,None,[],1,public,3773851,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-07-01T00:00:00,2018-07-01T00:25:42,2018-07-01T00:25:43,NaN
0,"[{'id': 14038886, 'name': 'joliva_MachineLearn...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4944283, 'full_name': 'Junier Oliva', ...",https://kilthub.cmu.edu/articles/thesis/Distri...,Machine learning has made incredible advances ...,None,[],1,public,18600219,...,https://kilthub.cmu.edu/articles/thesis/Distri...,,,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-07-01T00:00:00,2019-01-16T19:30:34,2019-01-16T19:30:3

### Compare high level metadata extract with detailed extract

In [13]:
## Column names in the "full" initial API call
df_full.columns

Index(['id', 'title', 'doi', 'handle', 'url', 'published_date', 'thumb',
       'defined_type', 'defined_type_name', 'group_id', 'url_private_api',
       'url_public_api', 'url_private_html', 'url_public_html', 'timeline',
       'resource_title', 'resource_doi'],
      dtype='object')

In [14]:
## Column names in the "detailed" API call
df_detailed.columns

Index(['files', 'custom_fields', 'authors', 'figshare_url', 'description',
       'funding', 'funding_list', 'version', 'status', 'size', 'created_date',
       'modified_date', 'is_public', 'is_confidential', 'is_metadata_record',
       'confidential_reason', 'metadata_reason', 'tags', 'categories',
       'references', 'has_linked_file', 'citation', 'is_embargoed',
       'embargo_date', 'embargo_type', 'embargo_title', 'embargo_reason',
       'embargo_options', 'id', 'title', 'doi', 'handle', 'url',
       'published_date', 'thumb', 'defined_type', 'defined_type_name',
       'group_id', 'url_private_api', 'url_public_api', 'url_private_html',
       'url_public_html', 'resource_title', 'resource_doi', 'license.value',
       'license.name', 'license.url', 'timeline.posted',
       'timeline.firstOnline', 'timeline.revision', 'timeline.submission'],
      dtype='object')

In [15]:
## What do these have in common?
overlap = list(set(df_full).intersection(set(df_detailed)))
overlap

['title',
 'doi',
 'defined_type',
 'url_public_api',
 'published_date',
 'url_private_html',
 'id',
 'url_private_api',
 'thumb',
 'handle',
 'url',
 'defined_type_name',
 'resource_title',
 'group_id',
 'url_public_html',
 'resource_doi']

In [16]:
## Are they the same?
sorted(df_full.columns) == sorted(overlap)

False

In [17]:
## So which ones are in one but not the other?
sorted(df_full.columns) 

['defined_type',
 'defined_type_name',
 'doi',
 'group_id',
 'handle',
 'id',
 'published_date',
 'resource_doi',
 'resource_title',
 'thumb',
 'timeline',
 'title',
 'url',
 'url_private_api',
 'url_private_html',
 'url_public_api',
 'url_public_html']

In [18]:
sorted(overlap)

['defined_type',
 'defined_type_name',
 'doi',
 'group_id',
 'handle',
 'id',
 'published_date',
 'resource_doi',
 'resource_title',
 'thumb',
 'title',
 'url',
 'url_private_api',
 'url_private_html',
 'url_public_api',
 'url_public_html']

In [19]:
# timeline is not in detailed extract, but it's the only one that's not the same

### Combine full and detailed API extracts by object ID

In [20]:
## Keep just id and timeline from initial extract (all others are in detailed extract)
## CHECK IF TIMELINE VAR IS IN COLLECTIONS AND PROJECT OBJECTS
df_small = df_full[['id','timeline']]

df_small

,id,timeline
0,6716609,"{'posted': '2016-08-01T00:00:00', 'firstOnline..."
1,6714890,"{'posted': '2012-08-01T00:00:00', 'firstOnline..."
2,6720908,"{'posted': '2013-12-01T00:00:00', 'firstOnline..."
3,6720416,"{'posted': '2018-03-01T00:00:00', 'firstOnline..."
4,6720836,"{'posted': '2017-12-01T00:00:00', 'firstOnline..."
5,6720410,"{'posted': '2012-05-01T00:00:00', 'firstOnline..."
6,6720413,"{'posted': '2013-07-01T00:00:00', 'firstOnline..."
7,7553882,"{'posted': '2018-07-01T00:00:00', 'revision': ..."
8,8336783,"{'posted': '2019-06-27T15:57:04', 'revision': ..."
9,6720509,"{'posted': '2013-10-01T00:00:00', 'firstOnline..."


In [21]:
## Merge small version of "full" dataframe with "detailed" dataframe
df_all = pd.merge(df_small, df_detailed, on = "id", how = "inner")

df_all

,id,timeline,files,custom_fields,authors,figshare_url,description,funding,funding_list,version,...,url_public_html,resource_title,resource_doi,license.value,license.name,license.url,timeline.posted,timeline.firstOnline,timeline.revision,timeline.submission
0,6716609,"{'posted': '2016-08-01T00:00:00', 'firstOnline...","[{'id': 12249995, 'name': 'Expressive Collabor...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5464304, 'full_name': 'Guangyu Xia', ...",https://kilthub.cmu.edu/articles/thesis/Expres...,<p>Techniques of Artificial Intelligence and H...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Expres...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2016-08-01T00:00:00,2018-07-01T00:04:06,2018-07-01T00:04:07,NaN
1,6714890,"{'posted': '2012-08-01T00:00:00', 'firstOnline...","[{'id': 12247874, 'name': 'Attribute Learning ...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5364275, 'full_name': 'Edith L.M. Law'...",https://kilthub.cmu.edu/articles/thesis/Attrib...,<p>This thesis is centered around the problem ...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Attrib...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-08-01T00:00:00,2018-06-30T23:16:04,2018-06-30T23:16:05,NaN
2,6720908,"{'posted': '2013-12-01T00:00:00', 'firstOnline...","[{'id': 12255338, 'name': 'On Learning from Co...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5363849, 'full_name': 'Liang Xiong', ...",https://kilthub.cmu.edu/articles/thesis/On_Lea...,In many machine learning problems and applicat...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/On_Lea...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-12-01T00:00:00,2018-07-01T00:48:03,2018-07-01T00:48:04,NaN
3,6720416,"{'posted': '2018-03-01T00:00:00', 'firstOnline...","[{'id': 12254780, 'name': 'Learning with Stale...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4947985, 'full_name': 'Wei Dai', 'is_a...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>A fundamental assumption behind most machin...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-03-01T00:00:00,2018-07-01T00:25:52,2018-07-01T00:25:53,NaN
4,6720836,"{'posted': '2017-12-01T00:00:00', 'firstOnline...","[{'id': 12255260, 'name': 'New Paradigms and O...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5466287, 'full_name': 'Yu-Xiang Wang'...",https://kilthub.cmu.edu/articles/thesis/New_Pa...,Machine learning (ML) has become one of the mo...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/New_Pa...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2017-12-01T00:00:00,2018-07-01T00:44:53,2018-07-01T00:44:54,NaN
5,6720410,"{'posted': '2012-05-01T00:00:00', 'firstOnline...","[{'id': 12254774, 'name': 'Learning with Limit...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5360768, 'full_name': 'Yi Zhang', 'is...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>In many real-world applications of supervis...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-05-01T00:00:00,2018-07-01T00:25:35,2018-07-01T00:25:36,NaN
6,6720413,"{'posted': '2013-07-01T00:00:00', 'firstOnline...","[{'id': 12254777, 'name': 'Learning with Sparc...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362001, 'full_name': 'Xi Chen', 'is_...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>The development of modern information techn...,None,[],1,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-07-01T00:00:00,2018-07-01T00:25:42,2018-07-01T00:25:43,NaN
7,7553882,"{'posted': '2018-07-01T00:00:00', 'revision': ...","[{'id': 14038886, 'name': 'joliva_MachineLearn...","[{'name': 'Publisher Stateme

In [22]:
## List files associated with public articles (by ID)

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}/files

#looks like files are already pulled from article details, so don't need a separate call - all info should already be there
#at some point go back and confirm that there's no new info in files API call

## Remaining to do:
### handle pagination within 'full' search